### **DLT PIPELINE**

In [0]:
import dlt
from pyspark.sql.functions import *

### **Streaming Table**

In [0]:
#Expectations
my_rules={
    "rule1" : "product_id IS NOT NULL",
    "rule2" : "product_name IS NOT NULL"
}

In [0]:
@dlt.table(name = "DimProducts_stage")

@dlt.expect_all_or_drop(my_rules)
def DimProducts_stage():
    #df = spark.readStream.table("databricks_cata.silver.products_silver")
    
    #return df

    df = spark.readStream.format("delta")\
        .option("skipChangeCommits", "true")\
        .load("abfss://silver@databricksetehimanshi.dfs.core.windows.net/products")
    return df

### **Streaming View**

In [0]:
@dlt.view

def DimProducts_view():
    df = spark.readStream.table("Live.DimProducts_stage")
    return df

### **DimProducts**

In [0]:
dlt.create_streaming_table("DimProducts")

In [0]:
dlt.apply_changes(
    target="DimProducts",
    source="Live.DimProducts_view",
    keys=["product_id"],
    sequence_by="product_id",
    stored_as_scd_type=2
)